In [28]:
#Nama:Muhammad Ilham Syadzili 
#Nim:221011403328
#Kelas: 05TPLE017 CS
#Mata Kuliah: Kecerdasan Buatan

import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

class KelayakanKreditFuzzy:
    def __init__(self):
        """Inisialisasi sistem fuzzy untuk penentuan kelayakan kredit."""
        self.setup_variables()
        self.setup_membership_functions()
        self.setup_rules()
        self.setup_control_system()

    def setup_variables(self):
        """Setup variabel fuzzy dengan range yang sesuai."""
        self.pendapatan = ctrl.Antecedent(np.arange(0, 20001, 1), 'pendapatan')
        self.utang = ctrl.Antecedent(np.arange(0, 10001, 1), 'utang')
        self.riwayat_kredit = ctrl.Antecedent(np.arange(0, 11, 1), 'riwayat_kredit')
        self.kelayakan = ctrl.Consequent(np.arange(0, 101, 1), 'kelayakan')

    def setup_membership_functions(self):
        """Definisi membership functions untuk setiap variabel."""
        self.pendapatan['rendah'] = fuzz.trapmf(self.pendapatan.universe, [0, 0, 3000, 6000])
        self.pendapatan['sedang'] = fuzz.trimf(self.pendapatan.universe, [4000, 10000, 16000])
        self.pendapatan['tinggi'] = fuzz.trapmf(self.pendapatan.universe, [12000, 15000, 20000, 20000])

        self.utang['rendah'] = fuzz.trapmf(self.utang.universe, [0, 0, 2000, 4000])
        self.utang['sedang'] = fuzz.trimf(self.utang.universe, [3000, 6000, 9000])
        self.utang['tinggi'] = fuzz.trapmf(self.utang.universe, [7000, 9000, 10000, 10000])

        self.riwayat_kredit['buruk'] = fuzz.trapmf(self.riwayat_kredit.universe, [0, 0, 2, 4])
        self.riwayat_kredit['sedang'] = fuzz.trimf(self.riwayat_kredit.universe, [3, 5, 7])
        self.riwayat_kredit['baik'] = fuzz.trapmf(self.riwayat_kredit.universe, [6, 8, 10, 10])

        self.kelayakan['tidak_layak'] = fuzz.trapmf(self.kelayakan.universe, [0, 0, 20, 40])
        self.kelayakan['layak'] = fuzz.trimf(self.kelayakan.universe, [30, 60, 90])
        self.kelayakan['sangat_layak'] = fuzz.trapmf(self.kelayakan.universe, [70, 90, 100, 100])

    def setup_rules(self):
        """Definisi aturan fuzzy."""
        self.rules = [
            ctrl.Rule(self.pendapatan['tinggi'] & self.utang['rendah'] & self.riwayat_kredit['baik'], 
                     self.kelayakan['sangat_layak']),
            ctrl.Rule(self.pendapatan['sedang'] & self.utang['rendah'] & self.riwayat_kredit['sedang'], 
                     self.kelayakan['layak']),
            ctrl.Rule(self.pendapatan['rendah'] | self.utang['tinggi'] | self.riwayat_kredit['buruk'], 
                     self.kelayakan['tidak_layak']),
            ctrl.Rule(self.pendapatan['tinggi'] & self.utang['sedang'] & self.riwayat_kredit['sedang'], 
                     self.kelayakan['layak']),
            ctrl.Rule(self.pendapatan['sedang'] & self.utang['sedang'] & self.riwayat_kredit['baik'], 
                     self.kelayakan['layak'])
        ]

    def setup_control_system(self):
        """Inisialisasi sistem kontrol fuzzy."""
        self.kelayakan_ctrl = ctrl.ControlSystem(self.rules)
        self.kelayakan_simulasi = ctrl.ControlSystemSimulation(self.kelayakan_ctrl)

    def validate_input(self, pendapatan, utang, riwayat_kredit):
        """Validasi input agar berada dalam range yang valid."""
        if not (0 <= pendapatan <= 20000):
            raise ValueError("Pendapatan harus berada antara 0 dan 20000")
        if not (0 <= utang <= 10000):
            raise ValueError("Utang harus berada antara 0 dan 10000")
        if not (0 <= riwayat_kredit <= 10):
            raise ValueError("Riwayat kredit harus berada antara 0 dan 10")

    def hitung_kelayakan(self, pendapatan, utang, riwayat_kredit):
        """
        Menghitung kelayakan kredit berdasarkan input yang diberikan.
        
        Parameters:
        -----------
        pendapatan : float
            Pendapatan bulanan (0-20000)
        utang : float
            Jumlah utang (0-10000)
        riwayat_kredit : float
            Skor riwayat kredit (0-10)
            
        Returns:
        --------
        dict
            Dictionary berisi hasil perhitungan kelayakan dan interpretasinya
        """
        try:
            self.validate_input(pendapatan, utang, riwayat_kredit)
            
            self.kelayakan_simulasi.input['pendapatan'] = pendapatan
            self.kelayakan_simulasi.input['utang'] = utang
            self.kelayakan_simulasi.input['riwayat_kredit'] = riwayat_kredit
            
            self.kelayakan_simulasi.compute()
            
            skor_kelayakan = self.kelayakan_simulasi.output['kelayakan']
            
            if skor_kelayakan < 40:
                interpretasi = "Tidak Layak"
            elif skor_kelayakan < 70:
                interpretasi = "Layak"
            else:
                interpretasi = "Sangat Layak"
                
            return {
                'skor': round(skor_kelayakan, 2),
                'interpretasi': interpretasi,
                'status': 'success'
            }
            
        except Exception as e:
            return {
                'status': 'error',
                'message': str(e)
            }

if __name__ == "__main__":
    sistem_kelayakan = KelayakanKreditFuzzy()
    
    # Test code
    test_cases = [
        #tes untuk kode yang benar
        {'pendapatan': 5000, 'utang': 8000, 'riwayat_kredit': 3},
        {'pendapatan': 20000, 'utang': 2000, 'riwayat_kredit': 9},
       #tes untuk kode error
        {'pendapatan': 2000000, 'utang': 20000, 'riwayat_kredit': 11}
    ]
    
    for case in test_cases:
        hasil = sistem_kelayakan.hitung_kelayakan(
            case['pendapatan'],
            case['utang'],
            case['riwayat_kredit']
        )
        
        if hasil['status'] == 'success':
            print(f"\nPendapatan: {case['pendapatan']}")
            print(f"Utang: {case['utang']}")
            print(f"Riwayat Kredit: {case['riwayat_kredit']}")
            print(f"Skor Kelayakan: {hasil['skor']}")
            print(f"Interpretasi: {hasil['interpretasi']}")
        else:
            print(f"\nError: {hasil['message']}")


Pendapatan: 5000
Utang: 8000
Riwayat Kredit: 3
Skor Kelayakan: 17.62
Interpretasi: Tidak Layak

Pendapatan: 20000
Utang: 2000
Riwayat Kredit: 9
Skor Kelayakan: 89.17
Interpretasi: Sangat Layak

Error: Pendapatan harus berada antara 0 dan 20000
